# Chain的主要概念

## 模型/IO封装

### 快速开始

In [19]:
# 请使用最新的open包
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 455.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 28.9 kB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.4.0
    Uninstalling tiktoken-0.4.0:
      Successfully uninstalled tiktoken-0.4.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.75
    Uninstalling langsmith-0.0.75:
      Successfully uninstalled langsmith-0.0.75
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.4
    Uninstalling langchain-core-0.1.4:
      Successfully uninstalled langchain-core-0.1.4


#### LLM

In [20]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

In [35]:
text = "请帮我想一想，生产彩色铅笔的公司有什么好名字?"
llm.invoke(text)

'\n\n1. 彩虹铅笔厂 \n2. 色彩铅笔工厂 \n3. 绚丽铅笔生产厂 \n4. 七彩铅笔公司 \n5. 彩绘铅笔制造厂 \n6. 色彩笔芯工坊 \n7. 色彩之星铅笔厂 \n8. 彩色艺术笔厂 \n9. 魔法彩铅笔厂 \n10. 色彩创意铅笔厂'

In [36]:
from langchain.schema import HumanMessage
messages = [HumanMessage(content=text)]
chat_model.invoke(messages)

AIMessage(content='1. 艳彩铅笔\n2. 彩绘铅笔\n3. 彩虹铅笔\n4. 色彩世界\n5. 彩铅工坊\n6. 炫彩铅笔\n7. 彩色艺术\n8. 色彩创意\n9. 彩虹创造\n10. 彩笔之家')

#### Prompt

In [34]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("请帮我想一想，生产{product}的公司有什么好名字?")
prompt.format(product="彩色铅笔")

'请帮我想一想，生产彩色铅笔的公司有什么好名字?'

In [26]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

#### Output parsers

In [43]:
from langchain.schema.output_parser import StrOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

#### 使用LCEL

In [32]:
template = "生成5个关于{text}的列表 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "colors"})

['red', 'blue', 'yellow', 'green', 'orange']

### 主要概念

#### 模型

请记住：OpenAI模型对提示语中包含JSON的情况非常友好。

#### 消息

- HumanMessage： 一般是纯文字内容
- AIMessage： 可能包含additional_kwargs，例如 funciton calling 提示
- SystemMessage：部份模型支持的内容提示
- FunctionMessage：函数调用的名称和参数
- ToolMessage：工具调用结果（与FunctionMessage不同）

#### 提示语

- PromptValue
- PromptTemplate
- MessagePromptTemplate
- MessagesPlaceholder
- ChatPromptTemplate

#### Output Parsers

- StrOutputParser：仅输出字符串；如果输出是 ChatModel，它会仅输出Message的content属性
- OpenAI Functions Parsers：处理OpenAI函数调用所需的函数名和参数
- Agent Output Parsers：帮助智能体解析执行计划

In [50]:
from langchain.schema.output_parser import StrOutputParser
parser = StrOutputParser()
response = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")
print(response)
print(parser.parse(response))

[HumanMessage(content='生成5个关于I love programming.的列表 I love programming..\n\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')]
[HumanMessage(content='生成5个关于I love programming.的列表 I love programming..\n\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')]


### Prompt封装

In [51]:
# 简单的例子
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("给我讲个关于{subject}的笑话")
print(template)
print(template.format(subject='小明'))

input_variables=['subject'] template='给我讲个关于{subject}的笑话'
给我讲个关于小明的笑话
